# Closest fit calculation comparison: brute force vs using clustering

This notebook compares the "closest fit" calculation between two different methods:  brute force, and using clustering.

## Definition: non-redundant nucleotide fragments

This calculation operates on the *non-redundant nucleotide fragments*, after 0.2 A clustering, e.g.
`./nucleotide-fragments/dinuc/AA.npy`. In the `.../origin/` subfolder, each fragment is annotated with a set of (one or more) origin PDB codes. From now on, these are simply called "fragments".

## Definition: closest fit calculation

For each fragment F, we try to find fragment FC, which is defined as the *closest-fitting fragment*: the fragment with the lowest RMSD from F. As an additional constraint, F and FC must come from different PDBs. Formally, either the origin set of F must have more than one member, or the origin set of FC must have at least one member that is not in the origin set of F.

The brute force approach to do this is: consider all candidate fragments that fulfill the origin criterion, calculate the superposition RMSD between F and all those candidates, and take the lowest one.


# Verify if the initial calculation has been done already

In [1]:
import os

dinuc_motifs = ["AA", "AC", "CA", "CC"]
trinuc_motifs = ["AAA", "AAC", "ACA", "ACC", "CAA", "CAC", "CCA", "CCC"]

ok = True
for lib, motifs in (("dinuc", dinuc_motifs), ("trinuc", trinuc_motifs)):
    for motif in motifs:
        filename = f"nucleotide-fragments/closest-fit/{lib}-{motif}.txt"
        if not os.path.exists(filename):
            print(f"Brute force calculation not done: missing file '{filename}'")
            ok = False
            break
    if not ok:
        break
if ok:
    print("Brute force calculation has been done")

ok = True
for lib, motifs in (("dinuc", dinuc_motifs), ("trinuc", trinuc_motifs)):
    for motif in motifs:
        filename = f"output/closest-fit/{lib}-{motif}.txt"
        if not os.path.exists(filename):
            print(f"Cluster-based calculation not done: missing file '{filename}'")
            ok = False
            break
    if not ok:
        break
if ok:
    print("Cluster-based calculation has been done")    

Brute force calculation has been done
Cluster-based calculation has been done


# Instructions how to run the initial calculation yourself

If the closest fit calculation has not been previously computed, this is how to do it

## Instruction: brute force calculation

Brute force closest-fit calculation is done in the `nucleotide-fragments` repo, a submodule of the current repo.

The scripts are included in the repo, but their results (the `nucleotide-fragemnts/closest-fit` subfolder) are not. 

You can calculate the results yourself by running the following scripts:

- `python closest-fit.py $lib $motif $chunk`, where:

  - `$lib` is "dinuc" or "trinuc"
  - `$motif` is every combination of A and C of the proper length,
    e.g. `CA` for a dinuc and `AAC` for a trinuc.
  - `$chunk` goes from 1 to 100

  An output file is generated in `closest-fit`

  For one particular value of `lib` `motif` `chunk`, the script takes three or four minutes on 8 cores. The whole computation is in the order of 500 CPU hours.

- `closest-fit-combine.sh` . This takes all output files for all `lib` `motif` `chunk` combinations, and generates one output file per motif: e.g. `closest-fit/dinuc-AA.txt`, `closest-fit/trinuc-CCC.txt`. This script runs instantly.

## Instruction: clustering-based calculation

Clustering-based closest-fit calculation is done in the current repo.

The scripts are included in the repo, but their results (`output/closest-fit/`) are not.

The script to run is `closest-fit.sh`, which does 12 invocations of `closest-fit.py`,
each generating one of the files in `output/closest-fit/`. Each invocation takes about 1 hour on 8 cores.


# Definition of the clustering-based method

For each fragment F, we try to identify one or more "bullseye" clusters where the closest-fitting fragment FC must surely be a member of. 

If successful, this provides us with a list of candidates for FC. Alternatively, if we can't find a bullseye cluster, we must build this list from multiple clusters. 

Finally, we calculate the RMSD of F towards every candidate, selecting the lowest.

## Identifying bullseye clusters


Bullseye clusters have the following properties:

- At least one of the members has a different PDB origin than F
- They satisfy the inequality X <= R, where:

  - X is the distance between the cluster heart and FC
  - R is the cluster radius

For each cluster, we try to prove that X <= R by exploiting the triangle inequality X <= Y + Z, where:

  - Y is the RMSD of the cluster heart to the fragment F
  - Z is the RMSD of the fragment F to FC

For each cluster, we compute Y. 

We don't know Z, but the computation of Y gives us an upper bound ZZ, which may get lower and lower while we search among the clusters.

We search first among R=2A clusters, and if it fails, among R=1A clusters. We search first the cluster that is closest in RMSD, then the other clusters.

We consider the search complete if we find at least one a bullseye cluster that is *small*, i.e. 50 members or less. While we search, we store all large bullseye clusters, as well as the computed values of Y for all small clusters. Whenever ZZ gets updated, we recompute Y + ZZ for all stored small clusters, in search of a hit where the result is smaller than R.

If we can't find any small bullseye cluster, we try once more to reduce ZZ by fitting on the first 50 members of every large bullseye cluster.

Once the search has terminated, if at least one bullseye cluster has been found, a candidate list is returned, consisting of the fragments that are part of *all* bullseye clusters (large and small), i.e. the intersection of the bullseye cluster members.

## Building a filtered list of candidates from multiple clusters

If no candidate list is returned by the bullseye cluster search, we must build a candidate list from multiple clusters. This is done as follows.

Initially, the candidate list contains all other fragments from a different PDB (which would be a brute force search).

We then consider all big 1A and 2A clusters (with at least 20 members), and try to eliminate their members.

To do so, first we get an upper estimate ZZ for Z (the RMSD between F and FC) by calculating Y, the RMSD towards the cluster heart of every big cluster, and taking the smallest value. If F is a member of one or more small 1A clusters, their cluster hearts are also considered for ZZ.

We then filter out in bulk all big 1A (i.e. R=1) clusters where Y > ZZ + R.

Finally, for each big 2A cluster, we look for clusters where the heart is much farther than ZZ. In that case, we can eliminate all members that are too close to the cluster heart. Formally, we eliminate from each 2A cluster all members for which Q < Y - ZZ, where Q is the RMSD of the member towards the cluster heart, a quantity that we have pre-computed.


# Comparison of the two closest-fit methods

In [2]:
import os
import numpy as np

dinuc_motifs = ["AA", "AC", "CA", "CC"]
trinuc_motifs = ["AAA", "AAC", "ACA", "ACC", "CAA", "CAC", "CCA", "CCC"]

for lib, motifs in (("dinuc", dinuc_motifs), ("trinuc", trinuc_motifs)):
    nfrags = 0
    any_different = False
    for motif in motifs:
        filename1 = f"nucleotide-fragments/closest-fit/{lib}-{motif}.txt"
        data1 = np.loadtxt(filename1)
        if data1.ndim != 2 or data1.shape[1] != 2:
            raise RuntimeError(f"Format error in file '{filename1}'")
        filename2 = f"output/closest-fit/{lib}-{motif}.txt"
        data2 = np.loadtxt(filename2)
        if data2.ndim != 2 or data2.shape[1] != 2:
            raise RuntimeError(f"Format error in file '{filename2}'")
        if len(data1) != len(data2):
            raise RuntimeError(f"Library {lib} motif {motif}: mismatch in number of fragments")
        nfrags += len(data1)
        different = (np.isclose(data1, data2, atol=1e-03).sum(axis=1) < 2)
        ndifferent = different.sum()
        if ndifferent:
            print(f"Library {lib}, motif {motif}: {ndifferent} differences")
            any_different = True
    if not any_different:
        print(f"Library {lib}: no differences")

Library dinuc: no differences
Library trinuc, motif CCC: 1 differences
